In [1]:
import os
import numpy as np

In [2]:
sys.path.append('/Users/nunofernandes/PycharmProjects/challenge_vc')

In [ ]:
from non_dominant_hand import analyze_disappearance_and_position
from end import analyze_single_hand_frames
from start import detect_non_dominant_hand_movement_
import numpy as np
from VGG19_helper import *
from main_helper import *
from keras.models import load_model
from frame_differencing import detect_poke_pen_lines
from probe_poke_phases import detect_poke_phase
from pen_phase import detect_pen_phase
from Annotate import *

In [260]:
def main(txt_dir='/Users/nunofernandes/PycharmProjects/challenge_vc/runs/detect/predict4/labels',model_path = '/Users/nunofernandes/PycharmProjects/challenge_vc/THIS_model.hdf5', input_folder = '/Users/nunofernandes/PycharmProjects/challenge_vc/frames_5_xyz_w', output_path = "/Users/nunofernandes/PycharmProjects/challenge_vc/Annotations_main"):
    """
    Main function that uses initialized variables.
    1. Detect at each trial Dominant/Non-Dominant Hand
    """
    processor = FrameProcessor()
    
    variables = initialize_variables(txt_dir, model_path, input_folder, output_path)
    
    metrics = variables['metrics']
    frame_numbers = variables['frame_numbers']
    last_valid_frame = variables['last_valid_frame']
    phases = variables['phases']
    end_frames = variables['end_frames']
    durations = variables['durations']
    non_dominant_hand_detections = variables['non_dominant_hand_detections']
    heights_non_dominant = variables['heights_non_dominant']
    x_centers_non_dominant = variables['x_centers_non_dominant']
    y_centers_non_dominant = variables['y_centers_non_dominant']
    x_centers_dominant = variables['x_centers_dominant']
    y_centers_dominant = variables['y_centers_dominant']
    predictions = variables['predictions']
    previous_image = variables['previous_image']
    
    line_lengths= variables['line_lengths']
    thetas = variables['thetas']
    first_poke_x = variables['first_poke_x']
    first_poke_y = variables['first_poke_y']
    
    last_valid_pen_length = variables['last_valid_pen_length']
    first_pen_x = variables['first_pen_x']
    first_pen_y = variables['first_pen_y']
    
    line_lengths_pen = variables['line_lengths_pen']
    
    buffer = variables['buffer']
    #Load model
    model_poke_pen = variables['model_poke_pen']
    
    

    txt_files = variables['txt_files']
    for frame_number, txt_file in enumerate(txt_files):
    
        # Reset metrics
        metrics['poke'] = 0
        metrics['pen'] = 0
        frame_numbers.append(frame_number)
    
        detections = parse_txt_file(txt_file)
        hand_detections = [d for d in detections if d[0] == 0 or d[0] == 1]
        
                                        ##################      Detect Dominant Hand       #################
        #Annotations of 2 handes available
        if len(hand_detections) == 2:
            #detects which hand disappeares mores times from the fram (non_dominant_hand) and check is right or left each frame
            dominant_hand_position = analyze_disappearance_and_position(txt_dir)[0]
            dominant_hand_detection = determine_hand(hand_detections, dominant_hand_position, return_dominant=True)
            non_dominant_hand_detection = determine_hand(hand_detections, dominant_hand_position, return_dominant=False)
    
            if non_dominant_hand_detection is None:
                continue
            #store Annotations from YOLO
            current_x_center_dominant, current_y_center_dominant, current_height_dominant, current_width_dominant = get_hand_data(dominant_hand_detection)
            current_x_center_non_dominant, current_y_center_non_dominant, current_height_non_dominant, current_width_non_dominant = get_hand_data(non_dominant_hand_detection)
    
            if None in (current_x_center_non_dominant, current_y_center_non_dominant, current_height_non_dominant):
                continue
            
            #Crop hand region for VGG-19 predictions and stores in memory for line detection 
            image_path = os.path.join(input_folder, f"frame_{frame_number:05}.jpg")
            image_cropped = crop_hand_region(image_path, detections, width_reduction=0.8, height_reduction=0.6, move_factor=1)
            
                                ############### Detect Start Operation ###############
            #check if is a valid movement to start operation
            if phases['start'] <= phases['end']:
                phases['poke'] = 0
                phases['pen'] = 0
                if len(x_centers_non_dominant) > 0:
                    # Detects an initial large x movement then awaits for a height contraction and a small movement
                    detect_initial_movement_non_dominant(
                        current_x_center_non_dominant, current_y_center_non_dominant,
                        x_centers_non_dominant, y_centers_non_dominant,
                        heights_non_dominant, current_height_non_dominant,
                        non_dominant_hand_detections, frame_number, phases, metrics
                    )
                                  ################### Poke Pen Detections #############
            elif phases['start'] > phases['end']:
                metrics['duration'][0] += 1  # Update duration of the trial
                if current_y_center_non_dominant > 0.7:
                    pred = make_prediction_VGG19(model_poke_pen, image_path=image_cropped, size=(224, 224))
                    predictions.append(pred)
    
                    # Call detect_poke_phase function
                    phases, line_lengths, thetas, first_poke_x, first_poke_y = detect_poke_phase(
                        predictions, previous_image, image_cropped,
                        current_x_center_dominant, current_y_center_dominant,
                        phases, line_lengths, thetas, first_poke_x, first_poke_y
                    )
    
                    
                    # Call detect_pen_phase function
                    phases, line_lengths_pen, last_valid_pen_length, first_pen_x, first_pen_y = detect_pen_phase(
                        predictions, previous_image, image_cropped,
                        current_x_center_dominant, current_y_center_dominant,
                        phases, last_valid_pen_length, first_pen_x, first_pen_y,
                        line_lengths_pen
                    )

            # Append current data to lists
            x_centers_non_dominant.append(current_x_center_non_dominant)
            y_centers_non_dominant.append(current_y_center_non_dominant)
            heights_non_dominant.append(current_height_non_dominant)
            x_centers_dominant.append(current_x_center_dominant)
            y_centers_dominant.append(current_y_center_dominant)
            previous_image = image_cropped

        # Check if only one hand is detected
        if len(hand_detections) == 1:
            # Assume this hand is the non-dominant hand
            hand_detection = hand_detections[0]
        
            # Call the auxiliary function to analyze and update
            buffer, last_valid_frame = analyze_and_update_hand_movement(
                hand_detection,
                y_centers_non_dominant,
                frame_number,
                last_valid_frame,
                buffer,
                phases,
                end_frames,
                metrics,
                durations,
                non_dominant_hand_detections
            )
        
        input_folder = '/Users/nunofernandes/PycharmProjects/challenge_vc/frames_5_xyz_w'

        out_folder = "/Users/nunofernandes/PycharmProjects/challenge_vc/annotations_test/"
        output_path = os.path.join(out_folder, f"frame_{frame_number:05}.jpg")

        image_path = os.path.join(input_folder, f"frame_{frame_number:05}.jpg")
        processor.process_frame(image_path, detections, output_path, phases, frame_number, font_size=50)
